# **Merge Train Meteo + Google Trends**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot') 

import warnings 
warnings.filterwarnings ('ignore')

In [3]:
train_meteo = pd.read_csv("../data/processed/train_meteo_full.csv")
train_google = pd.read_csv("../data/processed/train.csv")
test_meteo = pd.read_csv("../data/processed/test_meteo_full.csv")
test_google = pd.read_csv("../data/processed/test.csv")

print("Train meteo shape:", train_meteo.shape)
print("Train google shape:", train_google.shape)
print("Test meteo shape:", test_meteo.shape)
print("Test google shape:", test_google.shape)

Train meteo shape: (9196, 31)
Train google shape: (9196, 13)
Test meteo shape: (2288, 30)
Test google shape: (2288, 12)


In [4]:
train_meteo.head()

,Id,week,region_code,region_name,TauxGrippe,tend,dd,ff,t,td,...,ht_neige,ssfrai,perssfrai,rr1,rr3,rr6,rr12,rr24,date,saison
0,5523,201152,42,ALSACE,66,-40.625000,203.571429,3.302679,278.522321,276.712500,...,0.0,0.0,-45.000000,0.064286,0.192857,0.478571,0.807143,1.564286,2011-12-26,Hiver
1,5524,201152,72,AQUITAINE,24,-35.446429,198.750000,3.697321,280.775893,278.746429,...,0.0,0.0,-43.663366,0.062500,0.194595,0.471429,0.807143,1.628571,2011-12-26,Hiver
2,5525,201152,83,AUVERGNE,91,-36.964286,187.142857,2.183929,279.441071,276.532143,...,0.0,0.0,-45.272727,0.025000,0.085714,0.142857,0.357143,0.742857,2011-12-26,Hiver
3,5526,201152,25,BASSE-NORMANDIE,49,-50.446429,214.732143,4.633929,280.709821,279.047321,...,0.0,0.0,-46.451613,0.100000,0.257658,0.371429,0.800000,0.916667,2011-12-26,Hiver
4,5527,201152,26,BOURGOGNE,33,-41.071429,199.017857,3.447321,278.709821,277.808929,...,0.0,0.0,-45.000000,0.125000,0.350000,0.992857,1.514286,2.864286,2011-12-26,Hiver


In [5]:
train_google.head()

,Id,week,region_name,TauxGrippe,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
0,5523,201152,ALSACE,66,448112,479905,526481,250849,146978,1852325,2,2,2
1,5524,201152,AQUITAINE,24,732120,765143,896030,516065,344875,3254233,2,2,2
2,5525,201152,AUVERGNE,91,294709,305778,375485,223830,150880,1350682,3,3,2
3,5526,201152,BASSE-NORMANDIE,49,354514,336201,401766,228888,154315,1475684,2,2,2
4,5527,201152,BOURGOGNE,33,372023,369737,450664,269109,181201,1642734,2,2,1


In [6]:
train_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           9196 non-null   int64  
 1   week         9196 non-null   int64  
 2   region_code  9196 non-null   int64  
 3   region_name  9196 non-null   object 
 4   TauxGrippe   9196 non-null   int64  
 5   tend         9196 non-null   float64
 6   dd           9196 non-null   float64
 7   ff           9196 non-null   float64
 8   t            9196 non-null   float64
 9   td           9196 non-null   float64
 10  u            9196 non-null   float64
 11  vv           9196 non-null   float64
 12  n            9196 non-null   float64
 13  nbas         9196 non-null   float64
 14  hbas         9196 non-null   float64
 15  pres         9196 non-null   float64
 16  tn12         9196 non-null   float64
 17  tx12         9196 non-null   float64
 18  tminsol      9196 non-null   float64
 19  rafper

In [7]:
train_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 13 columns):
 #   Column                                                                              Non-Null Count  Dtype 
---  ------                                                                              --------------  ----- 
 0   Id                                                                                  9196 non-null   int64 
 1   week                                                                                9196 non-null   int64 
 2   region_name                                                                         9196 non-null   object
 3   TauxGrippe                                                                          9196 non-null   int64 
 4   pop_0_19                                                                            9196 non-null   int64 
 5   pop_20_39                                                                           9196 non-null   int6

## Séléction des colonnes

In [8]:
cols_meteo = [
    "Id",
    "week",
    "region_code",
    "region_name",
    "TauxGrippe",
    "tend",
    "dd",
    "ff",
    "t",
    "td",
    "u",
    "vv",
    "n",
    "nbas",
    "hbas",
    "pres",
    "tn12",
    "tx12",
    "tminsol",
    "rafper",
    "per",
    "ht_neige",
    "ssfrai",
    "perssfrai",
    "rr1",
    "rr3",
    "rr6",
    "rr12",
    "rr24",
    "date",
    "saison"
]

train_meteo = train_meteo[cols_meteo]

In [9]:
cols_meteo_test = [
    "Id",
    "week",
    "region_code",
    "region_name",
    #"TauxGrippe",
    "tend",
    "dd",
    "ff",
    "t",
    "td",
    "u",
    "vv",
    "n",
    "nbas",
    "hbas",
    "pres",
    "tn12",
    "tx12",
    "tminsol",
    "rafper",
    "per",
    "ht_neige",
    "ssfrai",
    "perssfrai",
    "rr1",
    "rr3",
    "rr6",
    "rr12",
    "rr24",
    "date",
    "saison"
]

test_meteo = test_meteo[cols_meteo_test]

In [10]:
cols_google = [
    "Id",
    "week",
    "region_name",
    "TauxGrippe",
    "pop_0_19",
    "pop_20_39",
    "pop_40_59",
    "pop_60_74",
    "pop_75_plus",
    "pop_total",
    "requete_grippe",
    "requete_grippe_aviaire_vaccin",
    "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie"
]

train_google = train_google[cols_google]

In [11]:
cols_google_test = [
    "Id",
    "week",
    "region_name",
    "pop_0_19",
    "pop_20_39",
    "pop_40_59",
    "pop_60_74",
    "pop_75_plus",
    "pop_total",
    "requete_grippe",
    "requete_grippe_aviaire_vaccin",
    "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie"
]

test_google = test_google[cols_google_test]

In [12]:
train_full = train_meteo.merge(
    train_google,
    on=["region_name", "week"],
    how="inner",
    suffixes=("", "_google")
)

print(train_full.shape)

(9196, 42)


In [13]:
test_full = test_meteo.merge(
    test_google,
    on=["region_name", "week"],
    how="inner",
    suffixes=("", "_google")
)

print(test_full.shape)

(2288, 40)


In [14]:
train_full.head()

,Id,week,region_code,region_name,TauxGrippe,tend,dd,ff,t,td,...,TauxGrippe_google,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
0,5523,201152,42,ALSACE,66,-40.625000,203.571429,3.302679,278.522321,276.712500,...,66,448112,479905,526481,250849,146978,1852325,2,2,2
1,5524,201152,72,AQUITAINE,24,-35.446429,198.750000,3.697321,280.775893,278.746429,...,24,732120,765143,896030,516065,344875,3254233,2,2,2
2,5525,201152,83,AUVERGNE,91,-36.964286,187.142857,2.183929,279.441071,276.532143,...,91,294709,305778,375485,223830,150880,1350682,3,3,2
3,5526,201152,25,BASSE-NORMANDIE,49,-50.446429,214.732143,4.633929,280.709821,279.047321,...,49,354514,336201,401766,228888,154315,1475684,2,2,2
4,5527,201152,26,BOURGOGNE,33,-41.071429,199.017857,3.447321,278.709821,277.808929,...,33,372023,369737,450664,269109,181201,1642734,2,2,1


In [15]:
train_full.to_csv("../data/data_modelisation/train_full.csv", index=False)

In [16]:
test_full.to_csv("../data/data_modelisation/test_full.csv", index=False)

## Fin